# Chronos-2-PETSA Benchmarking

This notebook evaluates the Chronos-2-PETSA (Parameter-Efficient Test-Time Adaptation) model on zero-shot datasets.

In [ ]:
# Clone Repository
!git clone https://github.com/emanueleromito/voyagers-forecasting.git
%cd voyagers-forecasting

In [ ]:
# Install dependencies
!pip install -e .[dev]
!pip install gluonts transformers accelerate typer typer-config rich wandb datasets
!pip install --upgrade sympy

In [ ]:
import sys
import os
import torch
import wandb
import pandas as pd
from huggingface_hub import HfApi
from google.colab import userdata

sys.path.append(os.path.abspath("src"))

from scripts.evaluation.evaluate import eval_pipeline_and_save_results
from chronos2.pipeline import Chronos2Pipeline
from chronos2.extensions.petsa.petsa import ChronosPETSAWrapper, ChronosPETSAPipeline

In [ ]:
# --- Secrets ---
try:
    HF_TOKEN = userdata.get('HF_TOKEN')
    WANDB_KEY = userdata.get('wandb')
    wandb.login(key=WANDB_KEY)
except Exception as e:
    print(f"Could not load secrets: {e}")
    HF_TOKEN = None

# --- Configuration ---
RUN_NAME = "chronos2-petsa-benchmarking"
MODEL_ID = "voyagersnlppolito/chronos2-baseline" 
BATCH_SIZE = 1 # Must be 1 for instance-level adaptation
PETSA_RANK = 8
PETSA_ALPHA = 16.0
PETSA_STEPS = 5
PETSA_LR = 1e-3
SPARSE_RATIO = 0.8

In [ ]:
# Load Pretrained Model
print(f"Loading model: {MODEL_ID}")
pipeline = Chronos2Pipeline.from_pretrained(MODEL_ID, device_map="cuda" if torch.cuda.is_available() else "cpu", torch_dtype=torch.bfloat16)
model = pipeline.model

# Wrap with PETSA
print("Wrapping model with PETSA...")
petsa_wrapper = ChronosPETSAWrapper(model, lora_rank=PETSA_RANK, lora_alpha=PETSA_ALPHA)

# Wrap with PETSA Pipeline
petsa_pipeline = ChronosPETSAPipeline(petsa_wrapper)
print("PETSA Pipeline ready.")

In [ ]:
# Run Zero-Shot Evaluation
print("Starting Zero-Shot Evaluation with PETSA...")
ZERO_SHOT_RESULTS_PATH = "evaluation_results_petsa_zero_shot.csv"
ZERO_SHOT_CONFIG_PATH = "scripts/evaluation/configs/zero-shot.yaml"

if not os.path.exists(ZERO_SHOT_CONFIG_PATH):
    print(f"Config file not found at {ZERO_SHOT_CONFIG_PATH}. Please check the path.")
else:
    # Run evaluation
    eval_pipeline_and_save_results(
        pipeline=petsa_pipeline,
        config_path=ZERO_SHOT_CONFIG_PATH,
        metrics_path=ZERO_SHOT_RESULTS_PATH,
        model_id=RUN_NAME,
        batch_size=BATCH_SIZE,
        sparse_ratio=SPARSE_RATIO,
    )

    # Log to WandB
    if wandb.run is not None:
        print("Logging results to WandB...")
        try:
            results_df = pd.read_csv(ZERO_SHOT_RESULTS_PATH)
            wandb.log({"evaluation_results_petsa": wandb.Table(dataframe=results_df)})
            
            # Log aggregate metrics
            avg_mase = results_df["MASE"].mean()
            avg_wql = results_df["WQL"].mean()
            wandb.log({"eval/petsa_avg_mase": avg_mase, "eval/petsa_avg_wql": avg_wql})
            print("Logged results to WandB.")
        except Exception as e:
            print(f"Failed to log to WandB: {e}")

    # Push to Hub
    if HF_TOKEN:
        print("Uploading results to Hub...")
        api = HfApi()
        try:
            api.upload_file(
                path_or_fileobj=ZERO_SHOT_RESULTS_PATH,
                path_in_repo="evaluation_results_petsa_zero_shot.csv",
                repo_id=f"voyagersnlppolito/{RUN_NAME}",
                repo_type="model"
            )
            print("Results uploaded successfully.")
        except Exception as e:
            print(f"Failed to upload results: {e}")
